# Feistel

In [1]:
def compose_permutations(P1, P2, n): # return polytope for F2 \circ F1
    ieqs = []
    eqns = []
    for ieq in P1.inequalities_list():
        ieqs.append(ieq[:2*n+1] + [0]*n + ieq[-1:] + [0])
    for eqn in P1.equations_list():
        eqns.append(eqn[:2*n+1] + [0]*n + eqn[-1:] + [0])
    for ieq in P2.inequalities_list():
        ieqs.append(ieq[:1] + [0]*n + ieq[1:2*n+1] + [0] + ieq[-1:])
    for eqn in P2.equations_list():
        eqns.append(eqn[:1] + [0]*n + eqn[1:2*n+1] + [0] + eqn[-1:])
    M = [[matrix.zero(ZZ, i, j) for j in (n, n, n, 1, 1)] for i in (n, n, 1)]
    M[0][0] = matrix.identity(ZZ, n)
    M[1][2] = matrix.identity(ZZ, n)
    M[2][3] = matrix.identity(ZZ, 1)
    M[2][4] = matrix.identity(ZZ, 1)
    return Polyhedron(ieqs=ieqs, eqns = eqns).linear_transformation(block_matrix(M))

def eval_output_property(Pf, Po, n): # return polytop for v * A^F
    ieqs = []
    eqns = []
    for ieq in Pf.inequalities_list():
        ieqs.append(ieq + [0])
    for eqn in Pf.equations_list():
        eqns.append(eqn + [0])
    for ieq in Po.inequalities_list():
        ieqs.append(ieq[:1] + [0]*n + ieq[1:n+1] + [0] + ieq[-1:])
    for eqn in Po.equations_list():
        eqns.append(eqn[:1] + [0]*n + eqn[1:n+1] + [0] + eqn[-1:])
    M = [[matrix.zero(ZZ, i, j) for j in (n, n, 1, 1)] for i in (n, 1)]
    M[0][0] = matrix.identity(ZZ, n)
    M[1][2] = matrix.identity(ZZ, 1)
    M[1][3] = matrix.identity(ZZ, 1)
    return Polyhedron(ieqs=ieqs, eqns = eqns).linear_transformation(block_matrix(M))

def get_polytope_function(p, n, d): # model could still be improved when d < n-1 as the degree influences the results then
    ieqs = []
    for i in range(2*n):
        ieqs.append([0] + [0]*i + [1] + [0]*(2*n-i))
        ieqs.append([p-1] + [0]*i + [-1] + [0]*(2*n-i))
    ieqs.append([0]*(2*n+1) + [1]) # ordp is at least 0
    ieqs.append([0] + [-1]*n + [d]*n + [d*(p-1)]) # constraints from function
    ieqs.append([-1] + [0]*n + [1]*n + [0]) # special treatment for output character 1
    return Polyhedron(ieqs=ieqs).convex_hull(Polyhedron(vertices=[[0]*(2*n+1)]))

def get_polytope_addition(p):
    return Polyhedron(ieqs = [[p-1, -1, 0, 0, 0], [p-1, 0, -1, 0, 0], [1, 0, 0, 0, -1], [0, 0, 0, 0, 1], [0, 1, 1, 0, -p], [0, 1, 0, 0, -1], [0, 0, 1, 0, -1], [p-1, -1, -1, 0, p-1]], eqns = [[0, 1, 1, -1, -(p-1)]])
    
def get_polytope_copy(p):
    return get_polytope_addition(p).linear_transformation(Matrix(ZZ, 4, 4, [[0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 0]]))

def feistel(P, p): # only 2 branches with each 1 element in f_p
    # evaluate the left branch of the state on a function and add it to the right branch, than swap the two states
    # we extract the eqns and ieqs constraints of the copy, addition and function and stitch them together
    # order is ul, ur, wl, wr, vl, vr, e_F, e_add
    eqns = []
    ieqs = []
    Pc = get_polytope_copy(p)
    Pa = get_polytope_addition(p)
    for ieq in P.inequalities_list():
        ieqs.append([ieq[0], 0, 0, ieq[1], ieq[2], 0, 0, ieq[3], 0])
    for eq in P.equations_list():
        eqns.append([eq[0], 0, 0, eq[1], eq[2], 0, 0, eq[3], 0])
    for ieq in Pc.inequalities_list():
        ieqs.append([ieq[0], ieq[1], 0, ieq[2], 0, 0, ieq[3], 0, 0])
    for eq in Pc.equations_list():
        eqns.append([eq[0], eq[1], 0, eq[2], 0, 0, eq[3], 0, 0])
    for ieq in Pa.inequalities_list():
        ieqs.append([ieq[0], 0, ieq[1], 0, ieq[2], ieq[3], 0, 0, ieq[4]])
    for eq in Pa.equations_list():
        eqns.append([eq[0], 0, eq[1], 0, eq[2], eq[3], 0, 0, eq[4]])
    # order is left input weight, right input weight, left output weight, left input weight, divisiblity
    return Polyhedron(ieqs = ieqs, eqns = eqns).linear_transformation(Matrix(ZZ, 5, 8, [[1, 0, 0, 0, 0, 0, 0, 0],[0, 1, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0],[0, 0, 0, 0, 0, 0, 1, 1]]))

## Generic Feistel

In [2]:
def constraint_variables_in_polyhedron(model, P, vrs):
    assert(P.ambient_dimension() == len(vrs))
    vrs = (1, ) + tuple(vrs)
    for ieq in P.inequalities_list():
        model.add_constraint(sum(x*y for x, y in zip(vrs, ieq)) >= 0)
    for eqn in P.equations_list():
        model.add_constraint(sum(x*y for x, y in zip(vrs, eqn)) == 0)

def maximum_degree(Ps, n, p, e, r): # find p^r-degree of polynomial represented by polytope P embedded in dimension n (Ps is a list of polytopes at each rotation)
    assert(e > 0)
    assert(r > 0)
    assert(e % r == 0)
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = [model.new_variable(integer=True, nonnegative=True) for _ in range(n)]
    for i in range(n):
        model.set_max(digits_input_exponents[i], (p-1))
    rotations_input_exponents = [[sum(p**i*digits_input_exponents[k][(i-j) % e] for i in range(e)) for j in range(e)] for k in range(n)]
    for i in range(e):
        constraint_variables_in_polyhedron(model, Ps[i], [rotations_input_exponents[j][i] for j in range(n)])
    model.set_objective(sum(sum(sum(p**j*digits_input_exponents[k][r*i+j] for j in range(r)) for i in range(e/r)) for k in range(n)))
    return model.solve()

def maximum_degree_with_divisibility(Ps, n, p, e, r, max_divisibility): # find p^r-degree of polynomial represented by polytope P embedded in dimension n (Ps is a list of polytopes at each rotation)
    assert(e > 0)
    assert(r > 0)
    assert(e % r == 0)
    model = MixedIntegerLinearProgram('PPL', maximization=True)
    digits_input_exponents = [model.new_variable(integer=True, nonnegative=True) for _ in range(n)]
    for i in range(n):
        model.set_max(digits_input_exponents[i], (p-1))
    rotations_input_exponents = [[sum(p**i*digits_input_exponents[k][(i-j) % e] for i in range(e)) for j in range(e)] for k in range(n)]
    valuations = model.new_variable(integer=True, nonnegative=True)
    valuation = 0
    for i in range(e):
        constraint_variables_in_polyhedron(model, Ps[i], [rotations_input_exponents[j][i] for j in range(n)] + [valuations[i]])
        valuation += valuations[i]
    model.add_constraint(valuation <= max_divisibility)
    model.set_objective(sum(sum(sum(p**j*digits_input_exponents[k][r*i+j] for j in range(r)) for i in range(e/r)) for k in range(n)))
    return model.solve()

## 64-bit prime

In [3]:
# degree and density and properties mod p
p = 2**64-2**32+1
d = 3

pdiv = divisors(p-1)

P_f = get_polytope_function(p, 1, d)
P_round = feistel(P_f, p)
Po = (P_round & Polyhedron(eqns = [[0, 0, 0, 1, 0, 0]], ieqs=[[1, 0, 0, 0, -1, 0]])).linear_transformation(Matrix(ZZ, 3, 5, [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]]))

r = 1
degree = []
monomial_estimation = []
properties = []
while True:
    # we focus on the right branch, because this branch has the slower degree growth
    ## fix output property and divisibility
    Pred = (Po & Polyhedron(eqns=[[0, 0, 0, 1]])).linear_transformation(Matrix(ZZ, 2, 3, [[1, 0, 0], [0, 1, 0]]))
    ## estimate number of monomials
    monomial_estimation.append(Pred.integral_points_count(explicit_enumeration_threshold=0))
    ## compute degree
    degree.append(maximum_degree([Pred], 2, p, 1, 1))
    ## properties
    Pred = (Po & Polyhedron(eqns=[[0, 0, 0, 1], [0, 1, 0, 0]])).linear_transformation(Matrix(ZZ, 1, 3, [[0, 1, 0]]))
    prop = maximum_degree([Pred], 1, p, 1, 1)
    if prop == p-1:
        Pred = (Po & Polyhedron(eqns=[[0, 0, 0, 1], [1-p, 0, 1, 0]])).linear_transformation(Matrix(ZZ, 1, 3, [[1, 0, 0]]))
        prop = maximum_degree([Pred], 1, p, 1, 1)
        for g in pdiv:
            if g > prop:
                properties.append((g, p-1)) # left branch, right branch
                break
    else:
        for g in pdiv:
            if g > prop:
                properties.append((0, g))
                break
    ## maximal number of monomials of that degree
    d = degree[-1]
    nb_mons = binomial(2+d, 2)
    if d >= p:
        nb_mons -= 2*binomial(2+d-p, 2)
    print(r, float(monomial_estimation[-1]/nb_mons), d, properties[-1])
    if d >= 2*(p-1)-1:
        break
    r += 1
    Po = eval_output_property(P_round, Po, 2)

1 0.6666666666666666 1 (0, 1)
2 0.5 3 (0, 2)
3 0.4 9 (0, 4)
4 0.35714285714285715 27 (0, 10)
5 0.34146341463414637 81 (0, 30)
6 0.3360655737704918 243 (0, 85)
7 0.33424657534246577 729 (0, 255)
8 0.3336380255941499 2187 (0, 768)
9 0.33343492837549527 6561 (0, 2560)
10 0.3333672017882544 19683 (0, 7680)
11 0.33334462320067737 59049 (0, 20480)
12 0.33333709666493555 177147 (0, 61440)
13 0.33333458778192165 531441 (0, 196608)
14 0.33333375148338734 1594323 (0, 557056)
15 0.33333347271674296 4782969 (0, 1671168)
16 0.33333337979447636 14348907 (0, 5242880)
17 0.33333334882038174 43046721 (0, 15728640)
18 0.33333333849568286 129140163 (0, 44564480)
19 0.3333333350541165 387420489 (0, 133693440)
20 0.3333333339069277 1162261467 (0, 402653184)
21 0.3333333335245315 3486784401 (0, 1342177280)
22 0.33333333339706606 10460353203 (0, 4026531840)
23 0.33333333335457754 31381059609 (0, 10737418240)
24 0.33333333334041476 94143178827 (0, 32212254720)
25 0.3333333333356938 282429536481 (0, 1030807879

In [4]:
# degree and density and properties mod p^2
p = 2**64-2**32+1
d = 3

pdiv = divisors(p-1)
P_f = get_polytope_function(p, 1, d)
P_round = feistel(P_f, p)
Po = (P_round & Polyhedron(eqns = [[0, 0, 0, 1, 0, 0]], ieqs=[[1, 0, 0, 0, -1, 0]])).linear_transformation(Matrix(ZZ, 3, 5, [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]]))

r = 1
degree = []
monomial_estimation = []
properties = []
while True:
    # we focus on the right branch, because this branch has the slower degree growth
    ## fix output property and divisibility
    Pred = (Po & Polyhedron(ieqs=[[1, 0, 0, -1]]))
    ## estimate number of monomials
    monomial_estimation.append(Pred.linear_transformation(Matrix(ZZ, 2, 3, [[1, 0, 0], [0, 1, 0]])).integral_points_count(explicit_enumeration_threshold=0))
    ## compute degree
    degree.append(maximum_degree_with_divisibility([Pred], 2, p, 1, 1, 1))
    ## properties
    Pred = (Po & Polyhedron(eqns=[[0, 1, 0, 0]], ieqs=[[1, 0, 0, -1]])).linear_transformation(Matrix(ZZ, 2, 3, [[0, 1, 0], [0, 0, 1]]))
    prop = maximum_degree_with_divisibility([Pred], 1, p, 1, 1, 1)
    if prop == p-1:
        Pred = (Po & Polyhedron(eqns=[[1-p, 0, 1, 0]], ieqs=[[1, 0, 0, -1]])).linear_transformation(Matrix(ZZ, 2, 3, [[1, 0, 0], [0, 0, 1]]))
        prop = maximum_degree_with_divisibility([Pred], 1, p, 1, 1, 1)
        for g in pdiv:
            if g > prop:
                properties.append((g, p-1)) # left branch, right branch
                break
    else:
        for g in pdiv:
            if g > prop:
                properties.append((0, g))
                break
    if len(properties) != r:
        properties.append((p-1, p-1))
    ## estimate number of monomials
    d = degree[-1]
    nb_mons = binomial(2+d, 2)
    if d >= p:
        nb_mons -= 2*binomial(2+d-p, 2)
    print(r, float(monomial_estimation[-1]/nb_mons), d, properties[-1])
    if d >= 2*(p-1):
        break
    r += 1
    Po = eval_output_property(P_round, Po, 2)

1 0.6666666666666666 1 (0, 1)
2 0.8823529411764706 30744573449024307201 (4, 18446744069414584320)
3 1.0 36893488138829168640 (18446744069414584320, 18446744069414584320)


# Degree 2 extension over 31-bit prime

In [5]:
# degree and properties mod p with q-degrees
p = 2**31-2**24+1
e = 2
d = 3

pdiv = divisors(p**e-1)

P_f = get_polytope_function(p**e, 1, d)
P_round = feistel(P_f, p**e)
Pos = [(P_round & Polyhedron(eqns = [[0, 0, 0, 1, 0, 0]], ieqs=[[p**i, 0, 0, 0, -1, 0]])).linear_transformation(Matrix(ZZ, 3, 5, [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]])) for i in range(e)]

r = 1
degree = []
qproperties = []
while True:
    # we focus on the right branch, because this branch has the slower degree growth
    ## fix output property and divisibility
    Preds = [(Po & Polyhedron(eqns=[[0, 0, 0, 1]])).linear_transformation(Matrix(ZZ, 2, 3, [[1, 0, 0], [0, 1, 0]])) for Po in Pos]
    ## compute degree
    degree.append(maximum_degree(Preds, 2, p, e, e))
    ## properties
    Pred = [(Po & Polyhedron(eqns=[[0, 0, 0, 1], [0, 1, 0, 0]])).linear_transformation(Matrix(ZZ, 1, 3, [[0, 1, 0]])) for Po in Pos]
    prop = maximum_degree(Pred, 1, p, e, e)
    if prop == p**e-1:
        Pred = [(Po & Polyhedron(eqns=[[0, 0, 0, 1], [1-p**e, 0, 1, 0]])).linear_transformation(Matrix(ZZ, 1, 3, [[1, 0, 0]])) for Po in Pos]
        prop = maximum_degree(Pred, 1, p, e, e)
        for g in pdiv:
            if g > prop:
                qproperties.append((g, p**e-1)) # left branch, right branch
                break
    else:
        for g in pdiv:
            if g > prop:
                qproperties.append((0, g))
                break
    print(r, degree[-1], qproperties[-1])
    if degree[-1] >= 2*(p**e-1)-1:
        break
    r += 1
    Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]

1 1 (0, 1)
2 3 (0, 2)
3 9 (0, 4)
4 27 (0, 12)
5 81 (0, 32)
6 243 (0, 96)
7 729 (0, 254)
8 2187 (0, 762)
9 6561 (0, 2264)
10 19683 (0, 6792)
11 59049 (0, 24384)
12 177147 (0, 65024)
13 531441 (0, 195072)
14 1594323 (0, 575056)
15 4782969 (0, 1725168)
16 14348907 (0, 5019332)
17 43046721 (0, 15057996)
18 129140163 (0, 49938432)
19 387420489 (0, 133169152)
20 1162261467 (0, 399507456)
21 3486784401 (0, 1177714688)
22 10460353203 (0, 3533144064)
23 31381059609 (0, 11363767648)
24 94143178827 (0, 34091302944)
25 282429536481 (0, 113060610048)
26 847288609443 (0, 301494960128)
27 2541865828329 (0, 904484880384)
28 7625597484987 (0, 2611016351744)
29 22876792454961 (0, 7833049055232)
30 68630377364883 (0, 23091175860736)
31 205891132094649 (0, 69273527582208)
32 617673396283947 (0, 250657569767424)
33 1853020188851841 (0, 668420186046464)
34 5559060566555523 (0, 2005260558139392)
35 16677181699666569 (0, 5911341020348416)
36 50031545098999707 (0, 17734023061045248)
37 150094635296999121 (0, 7

In [6]:
# degree and properties mod p with p-degrees
p = 2**31-2**24+1
e = 2
d = 3

pdiv = [0] + divisors(p-1)

P_f = get_polytope_function(p**e, 1, d)
P_round = feistel(P_f, p**e)
Pos = [(P_round & Polyhedron(eqns = [[0, 0, 0, 1, 0, 0]], ieqs=[[p**i, 0, 0, 0, -1, 0]])).linear_transformation(Matrix(ZZ, 3, 5, [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]])) for i in range(e)]

r = 1
degree = []
pproperties = []
while True:
    # we focus on the right branch, because this branch has the slower degree growth
    ## fix output property and divisibility
    Preds = [(Po & Polyhedron(eqns=[[0, 0, 0, 1]])).linear_transformation(Matrix(ZZ, 2, 3, [[1, 0, 0], [0, 1, 0]])) for Po in Pos]
    ## compute degree
    degree.append(maximum_degree(Preds, 2, p, e, 1))
    ## properties
    Pred = [(Po & Polyhedron(eqns=[[0, 0, 0, 1], [0, 1, 0, 0]])).linear_transformation(Matrix(ZZ, 1, 3, [[0, 1, 0]])) for Po in Pos]
    prop = maximum_degree(Pred, 1, p, e, 1)
    if prop == e*(p-1):
        Pred = [(Po & Polyhedron(eqns=[[0, 0, 0, 1], [e*(1-p), 0, 1, 0]])).linear_transformation(Matrix(ZZ, 1, 3, [[1, 0, 0]])) for Po in Pos]
        prop = maximum_degree(Pred, 1, p, e, 1)
        res = []
        for i in range(e):
            for g in pdiv:
                if (sum(res) + g) > prop:
                    res.append(g)
                    break
            else:
                res.append(p-1)
        pproperties.append((tuple(res), (p-1,)*e))
    else:
        res = []
        for i in range(e):
            for g in pdiv:
                if (sum(res) + g) > prop:
                    res.append(g)
                    break
            else:
                res.append(p-1)
        pproperties.append(((0,)*e, tuple(res)))
    print(r, degree[-1], pproperties[-1])
    if degree[-1] >= 2*e*(p-1)-1:
        break
    r += 1
    Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]

1 1 ((0, 0), (1, 0))
2 3 ((0, 0), (2, 0))
3 9 ((0, 0), (4, 0))
4 27 ((0, 0), (16, 0))
5 81 ((0, 0), (32, 0))
6 243 ((0, 0), (127, 0))
7 729 ((0, 0), (254, 0))
8 2187 ((0, 0), (1016, 0))
9 6561 ((0, 0), (4064, 0))
10 19683 ((0, 0), (8128, 0))
11 59049 ((0, 0), (32512, 0))
12 177147 ((0, 0), (65024, 0))
13 531441 ((0, 0), (260096, 0))
14 1594323 ((0, 0), (1040384, 0))
15 4782969 ((0, 0), (2080768, 0))
16 14348907 ((0, 0), (8323072, 0))
17 43046721 ((0, 0), (16646144, 0))
18 129140163 ((0, 0), (66584576, 0))
19 387420489 ((0, 0), (133169152, 0))
20 1162261467 ((0, 0), (532676608, 0))
21 2582732421 ((0, 0), (2130706432, 0))
22 3701852718 ((0, 0), (2130706432, 1))
23 4074892826 ((0, 0), (2130706432, 4))
24 4199239555 ((0, 0), (2130706432, 16))
25 4240688543 ((0, 0), (2130706432, 64))
26 4254505108 ((0, 0), (2130706432, 254))
27 4259111336 ((0, 0), (2130706432, 508))
28 4260648867 ((0, 0), (2130706432, 2032))
29 4261167739 ((0, 0), (2130706432, 4064))
30 4261359785 ((0, 0), (2130706432, 1625

In [7]:
# for each round, check if p degree or q degree gives better properties
for r, (pprop, qprop) in enumerate(zip(pproperties, qproperties), start=1):
    pdata = (pprop[0][0]+1)*(pprop[0][1]+1)*(pprop[1][0]+1)*(pprop[1][1]+1)
    qdata = (qprop[0]+1)*(qprop[1]+1)
    if pdata < qdata:
        print(r, pprop)
    elif pdata == qdata:
        print(r, pprop, qprop)
    else:
        print(r, qprop)

1 ((0, 0), (1, 0)) (0, 1)
2 ((0, 0), (2, 0)) (0, 2)
3 ((0, 0), (4, 0)) (0, 4)
4 (0, 12)
5 ((0, 0), (32, 0)) (0, 32)
6 (0, 96)
7 ((0, 0), (254, 0)) (0, 254)
8 (0, 762)
9 (0, 2264)
10 (0, 6792)
11 (0, 24384)
12 ((0, 0), (65024, 0)) (0, 65024)
13 (0, 195072)
14 (0, 575056)
15 (0, 1725168)
16 (0, 5019332)
17 (0, 15057996)
18 (0, 49938432)
19 ((0, 0), (133169152, 0)) (0, 133169152)
20 (0, 399507456)
21 (0, 1177714688)
22 (0, 3533144064)
23 ((0, 0), (2130706432, 4))
24 (0, 34091302944)
25 (0, 113060610048)
26 (0, 301494960128)
27 (0, 904484880384)
28 (0, 2611016351744)
29 (0, 7833049055232)
30 (0, 23091175860736)
31 (0, 69273527582208)
32 (0, 250657569767424)
33 (0, 668420186046464)
34 (0, 2005260558139392)
35 (0, 5911341020348416)
36 (0, 17734023061045248)
37 (0, 70936092244180992)
38 (0, 189162912651149312)
39 (0, 567488737953447936)
40 (0, 1513303301209194496)
41 ((0, 0), (2130706432, 2130706432)) (0, 4539909903627583488)
42 ((2130706432, 2130706432), (2130706432, 2130706432)) (4539909903

In [8]:
# degree and properties mod p^2 with q-degrees
p = 2**31-2**24+1
e = 2
d = 3

pdiv = divisors(p**e-1)

P_f = get_polytope_function(p**e, 1, d)
P_round = feistel(P_f, p**e)
Pos = [(P_round & Polyhedron(eqns = [[0, 0, 0, 1, 0, 0]], ieqs=[[p**i, 0, 0, 0, -1, 0]])).linear_transformation(Matrix(ZZ, 3, 5, [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]])) for i in range(e)]

r = 1
degree = []
qproperties = []
while True:
    # we focus on the right branch, because this branch has the slower degree growth
    ## fix output property and divisibility
    Preds = [(Po & Polyhedron(ieqs=[[1, 0, 0, -1]])) for Po in Pos]
    ## compute degree
    degree.append(maximum_degree_with_divisibility(Preds, 2, p, e, e, 1))
    ## properties
    Pred = [(Po & Polyhedron(eqns=[[0, 1, 0, 0]], ieqs=[[1, 0, 0, -1]])).linear_transformation(Matrix(ZZ, 2, 3, [[0, 1, 0], [0, 0, 1]])) for Po in Pos]
    prop = maximum_degree_with_divisibility(Pred, 1, p, e, e, 1)
    if prop == p**e-1:
        Pred = [(Po & Polyhedron(eqns=[[1-p**e, 0, 1, 0]], ieqs=[[1, 0, 0, -1]])).linear_transformation(Matrix(ZZ, 2, 3, [[1, 0, 0], [0, 0, 1]])) for Po in Pos]
        prop = maximum_degree_with_divisibility(Pred, 1, p, e, e, 1)
        for g in pdiv:
            if g > prop:
                qproperties.append((g, p**e-1)) # left branch, right branch
                break
    else:
        for g in pdiv:
            if g > prop:
                qproperties.append((0, g))
                break
    print(r, degree[-1], qproperties[-1])
    if degree[-1] >= 2*(p**e-1)-1:
        break
    r += 1
    Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]

1 1 (0, 1)
2 7566516503205030571 (0, 4539909903627583488)
3 9079819802993754117 (0, 4539909903627583488)
4 9079819802993754135 (0, 4539909903627583488)
5 9079819802993754189 (0, 4539909903627583488)
6 9079819802993754351 (0, 4539909903627583488)
7 9079819802993754837 (0, 4539909903627583488)
8 9079819802993756295 (0, 4539909903627583488)
9 9079819802993760669 (0, 4539909903627583488)
10 9079819802993773791 (0, 4539909903627583488)
11 9079819802993813157 (0, 4539909903627583488)
12 9079819802993931255 (0, 4539909903627583488)
13 9079819802994285549 (0, 4539909903627583488)
14 9079819802995348431 (0, 4539909903627583488)
15 9079819802998537077 (0, 4539909903627583488)
16 9079819803008103015 (0, 4539909903627583488)
17 9079819803036800829 (0, 4539909903627583488)
18 9079819803122894271 (0, 4539909903627583488)
19 9079819803381174597 (0, 4539909903627583488)
20 9079819804156015575 (0, 4539909903627583488)
21 9079819805576486532 (0, 4539909903627583488)
22 9079819806695606828 (4539909903627

In [9]:
# degree and density and properties mod p^2 with p-degrees
p = 2**31-2**24+1
e = 2
d = 3

pdiv = [0] + divisors(p-1)

P_f = get_polytope_function(p**e, 1, d)
P_round = feistel(P_f, p**e)
Pos = [(P_round & Polyhedron(eqns = [[0, 0, 0, 1, 0, 0]], ieqs=[[p**i, 0, 0, 0, -1, 0]])).linear_transformation(Matrix(ZZ, 3, 5, [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]])) for i in range(e)]
Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]
Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]
Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]
r = 4 # skipping the first 3 rounds do to problems with the PPL solver
degree = []
pproperties = []
while True:
    # we focus on the right branch, because this branch has the slower degree growth
    ## fix output property and divisibility
    Preds = [(Po & Polyhedron(ieqs=[[1, 0, 0, -1]])) for Po in Pos]
    ## compute degree
    degree.append(maximum_degree_with_divisibility(Preds, 2, p, e, 1, 1))
    ## properties
    Pred = [(Po & Polyhedron(eqns=[[0, 1, 0, 0]], ieqs=[[1, 0, 0, -1]])).linear_transformation(Matrix(ZZ, 2, 3, [[0, 1, 0], [0, 0, 1]])) for Po in Pos]
    prop = maximum_degree_with_divisibility(Pred, 1, p, e, 1, 1)
    if prop == e*(p-1):
        Pred = [(Po & Polyhedron(eqns=[[e*(1-p), 0, 1, 0]], ieqs=[[1, 0, 0, -1]])).linear_transformation(Matrix(ZZ, 2, 3, [[1, 0, 0], [0, 0, 1]])) for Po in Pos]
        prop = maximum_degree_with_divisibility(Pred, 1, p, e, 1, 1)
        res = []
        for i in range(e):
            for g in pdiv:
                if (sum(res) + g) > prop:
                    res.append(g)
                    break
            else:
                res.append(p-1)
        pproperties.append((tuple(res), (p-1,)*e))
    else:
        res = []
        for i in range(e):
            for g in pdiv:
                if (sum(res) + g) > prop:
                    res.append(g)
                    break
            else:
                res.append(p-1)
        pproperties.append(((0,)*e, tuple(res)))
    print(r, degree[-1], pproperties[-1])
    if degree[-1] >= 2*e*(p-1)-1:
        break
    r += 1
    Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]

4 4261412887 ((0, 0), (2130706432, 16))
5 4261412941 ((0, 0), (2130706432, 32))
6 4261413103 ((0, 0), (2130706432, 127))
7 4261413589 ((0, 0), (2130706432, 254))
8 4261415047 ((0, 0), (2130706432, 1016))
9 4261419421 ((0, 0), (2130706432, 4064))
10 4261432543 ((0, 0), (2130706432, 8128))
11 4261471909 ((0, 0), (2130706432, 32512))
12 4261590007 ((0, 0), (2130706432, 65024))
13 4261944301 ((0, 0), (2130706432, 260096))
14 4263007183 ((0, 0), (2130706432, 1040384))
15 4266195829 ((0, 0), (2130706432, 2080768))
16 4275761767 ((0, 0), (2130706432, 8323072))
17 4304459581 ((0, 0), (2130706432, 16646144))
18 4390553023 ((0, 0), (2130706432, 66584576))
19 4648833349 ((0, 0), (2130706432, 133169152))
20 5423674327 ((0, 0), (2130706432, 532676608))
21 6844145284 ((0, 0), (2130706432, 2130706432))
22 7963265580 ((2130706432, 2130706432), (2130706432, 2130706432))
23 8336305678 ((2130706432, 2130706432), (2130706432, 2130706432))
24 8460652378 ((2130706432, 2130706432), (2130706432, 2130706432))


In [10]:
# for each round, check if p degree or q degree gives better properties
for r, (pprop, qprop) in enumerate(zip(pproperties, qproperties[3:]), start=4):
    pdata = (pprop[0][0]+1)*(pprop[0][1]+1)*(pprop[1][0]+1)*(pprop[1][1]+1)
    qdata = (qprop[0]+1)*(qprop[1]+1)
    if pdata < qdata:
        print(r, pprop)
    elif pdata == qdata:
        print(r, pprop, qprop)
    else:
        print(r, qprop)

4 ((0, 0), (2130706432, 16))
5 ((0, 0), (2130706432, 32))
6 ((0, 0), (2130706432, 127))
7 ((0, 0), (2130706432, 254))
8 ((0, 0), (2130706432, 1016))
9 ((0, 0), (2130706432, 4064))
10 ((0, 0), (2130706432, 8128))
11 ((0, 0), (2130706432, 32512))
12 ((0, 0), (2130706432, 65024))
13 ((0, 0), (2130706432, 260096))
14 ((0, 0), (2130706432, 1040384))
15 ((0, 0), (2130706432, 2080768))
16 ((0, 0), (2130706432, 8323072))
17 ((0, 0), (2130706432, 16646144))
18 ((0, 0), (2130706432, 66584576))
19 ((0, 0), (2130706432, 133169152))
20 ((0, 0), (2130706432, 532676608))
21 ((0, 0), (2130706432, 2130706432)) (0, 4539909903627583488)
22 ((2130706432, 2130706432), (2130706432, 2130706432)) (4539909903627583488, 4539909903627583488)
23 ((2130706432, 2130706432), (2130706432, 2130706432)) (4539909903627583488, 4539909903627583488)
24 ((2130706432, 2130706432), (2130706432, 2130706432)) (4539909903627583488, 4539909903627583488)
25 ((2130706432, 2130706432), (2130706432, 2130706432)) (4539909903627583488,

# Degree 4 extension over 17-bit prime

In [11]:
# degree and properties mod p with q-degrees
p = 2**17-1
e = 4
d = 3

pdiv = divisors(p**e-1)

P_f = get_polytope_function(p**e, 1, d)
P_round = feistel(P_f, p**e)
Pos = [(P_round & Polyhedron(eqns = [[0, 0, 0, 1, 0, 0]], ieqs=[[p**i, 0, 0, 0, -1, 0]])).linear_transformation(Matrix(ZZ, 3, 5, [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]])) for i in range(e)]

r = 1
degree = []
qproperties = []
while True:
    # we focus on the right branch, because this branch has the slower degree growth
    ## fix output property and divisibility
    Preds = [(Po & Polyhedron(eqns=[[0, 0, 0, 1]])).linear_transformation(Matrix(ZZ, 2, 3, [[1, 0, 0], [0, 1, 0]])) for Po in Pos]
    ## compute degree
    degree.append(maximum_degree(Preds, 2, p, e, e))
    ## properties
    Pred = [(Po & Polyhedron(eqns=[[0, 0, 0, 1], [0, 1, 0, 0]])).linear_transformation(Matrix(ZZ, 1, 3, [[0, 1, 0]])) for Po in Pos]
    prop = maximum_degree(Pred, 1, p, e, e)
    if prop == p**e-1:
        Pred = [(Po & Polyhedron(eqns=[[0, 0, 0, 1], [1-p**e, 0, 1, 0]])).linear_transformation(Matrix(ZZ, 1, 3, [[1, 0, 0]])) for Po in Pos]
        prop = maximum_degree(Pred, 1, p, e, e)
        for g in pdiv:
            if g > prop:
                qproperties.append((g, p**e-1)) # left branch, right branch
                break
    else:
        for g in pdiv:
            if g > prop:
                qproperties.append((0, g))
                break
    print(r, degree[-1], qproperties[-1])
    if degree[-1] >= 2*(p**e-1)-1:
        break
    r += 1
    Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]

1 1 (0, 1)
2 3 (0, 2)
3 9 (0, 4)
4 27 (0, 10)
5 81 (0, 30)
6 243 (0, 85)
7 729 (0, 255)
8 2187 (0, 768)
9 6561 (0, 2210)
10 19683 (0, 6630)
11 59049 (0, 19968)
12 177147 (0, 61440)
13 531441 (0, 179605)
14 1594323 (0, 532480)
15 4782969 (0, 1597440)
16 14348907 (0, 5003344)
17 43046721 (0, 14483456)
18 129140163 (0, 43274240)
19 387420489 (0, 129822720)
20 1162261467 (0, 390260832)
21 3486784401 (0, 1163202560)
22 10460353203 (0, 3489607680)
23 31381059609 (0, 10572065872)
24 94143178827 (0, 31716197616)
25 282429536481 (0, 94164746240)
26 847288609443 (0, 282494238720)
27 2541865828329 (0, 849069040345)
28 7625597484987 (0, 2547207121035)
29 22876792454961 (0, 7865110429696)
30 68630377364883 (0, 23004815337472)
31 205891132094649 (0, 69014446012416)
32 617673396283947 (0, 210675205406720)
33 1853020188851841 (0, 632025616220160)
34 5559060566555523 (0, 2054083252715520)
35 16677181699666569 (0, 5564458862804992)
36 50031545098999707 (0, 16693376588414976)
37 150094635296999121 (0, 54

In [14]:
# degree and properties mod p with p^2-degrees
p = 2**17-1
e = 4
d = 3
s = 2
pdiv = [0] + divisors(p**s-1)

P_f = get_polytope_function(p**e, 1, d)
P_round = feistel(P_f, p**e)
Pos = [(P_round & Polyhedron(eqns = [[0, 0, 0, 1, 0, 0]], ieqs=[[p**i, 0, 0, 0, -1, 0]])).linear_transformation(Matrix(ZZ, 3, 5, [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]])) for i in range(e)]

r = 1
degree = []
p2properties = []
while True:
    # we focus on the right branch, because this branch has the slower degree growth
    ## fix output property and divisibility
    Preds = [(Po & Polyhedron(eqns=[[0, 0, 0, 1]])).linear_transformation(Matrix(ZZ, 2, 3, [[1, 0, 0], [0, 1, 0]])) for Po in Pos]
    ## compute degree
    degree.append(maximum_degree(Preds, 2, p, e, s))
    ## properties
    Pred = [(Po & Polyhedron(eqns=[[0, 0, 0, 1], [0, 1, 0, 0]])).linear_transformation(Matrix(ZZ, 1, 3, [[0, 1, 0]])) for Po in Pos]
    prop = maximum_degree(Pred, 1, p, e, s)
    if prop == e/s*(p**s-1):
        Pred = [(Po & Polyhedron(eqns=[[0, 0, 0, 1], [e/s*(1-p**s), 0, 1, 0]])).linear_transformation(Matrix(ZZ, 1, 3, [[1, 0, 0]])) for Po in Pos]
        prop = maximum_degree(Pred, 1, p, e, s)
        res = []
        for i in range(e/s):
            for g in pdiv:
                if (sum(res) + g) > prop:
                    res.append(g)
                    break
            else:
                res.append(p**s-1)
        p2properties.append((tuple(res), (p**s-1,)*(e//s)))
    else:
        res = []
        for i in range(e/s):
            for g in pdiv:
                if (sum(res) + g) > prop:
                    res.append(g)
                    break
            else:
                res.append(p**s-1)
        p2properties.append(((0,)*(e//s), tuple(res)))
    print(r, degree[-1], p2properties[-1])
    if degree[-1] >= 2*e/s*(p**s-1)-1:
        break
    r += 1
    Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]

1 1 ((0, 0), (1, 0))
2 3 ((0, 0), (2, 0))
3 9 ((0, 0), (4, 0))
4 27 ((0, 0), (10, 0))
5 81 ((0, 0), (30, 0))
6 243 ((0, 0), (85, 0))
7 729 ((0, 0), (255, 0))
8 2187 ((0, 0), (768, 0))
9 6561 ((0, 0), (2560, 0))
10 19683 ((0, 0), (7680, 0))
11 59049 ((0, 0), (20480, 0))
12 177147 ((0, 0), (61440, 0))
13 531441 ((0, 0), (196608, 0))
14 1594323 ((0, 0), (557056, 0))
15 4782969 ((0, 0), (1671168, 0))
16 14348907 ((0, 0), (5263360, 0))
17 43046721 ((0, 0), (15790080, 0))
18 129140163 ((0, 0), (44738560, 0))
19 387420489 ((0, 0), (134215680, 0))
20 1162261467 ((0, 0), (429490176, 0))
21 3486784401 ((0, 0), (1431633920, 0))
22 10460353203 ((0, 0), (4294901760, 0))
23 21913424563 ((0, 0), (17179607040, 0))
24 30210617577 ((0, 0), (17179607040, 1))
25 32976348591 ((0, 0), (17179607040, 5))
26 33898258958 ((0, 0), (17179607040, 16))
27 34205562501 ((0, 0), (17179607040, 51))
28 34307997279 ((0, 0), (17179607040, 160))
29 34342142994 ((0, 0), (17179607040, 480))
30 34353527266 ((0, 0), (171796070

In [15]:
# degree and properties mod p with p-degrees
p = 2**17-1
e = 4
d = 3

pdiv = [0] + divisors(p-1)

P_f = get_polytope_function(p**e, 1, d)
P_round = feistel(P_f, p**e)
Pos = [(P_round & Polyhedron(eqns = [[0, 0, 0, 1, 0, 0]], ieqs=[[p**i, 0, 0, 0, -1, 0]])).linear_transformation(Matrix(ZZ, 3, 5, [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]])) for i in range(e)]

r = 1
degree = []
pproperties = []
while True:
    # we focus on the right branch, because this branch has the slower degree growth
    ## fix output property and divisibility
    Preds = [(Po & Polyhedron(eqns=[[0, 0, 0, 1]])).linear_transformation(Matrix(ZZ, 2, 3, [[1, 0, 0], [0, 1, 0]])) for Po in Pos]
    ## compute degree
    degree.append(maximum_degree(Preds, 2, p, e, 1))
    ## properties
    Pred = [(Po & Polyhedron(eqns=[[0, 0, 0, 1], [0, 1, 0, 0]])).linear_transformation(Matrix(ZZ, 1, 3, [[0, 1, 0]])) for Po in Pos]
    prop = maximum_degree(Pred, 1, p, e, 1)
    if prop == e*(p-1):
        Pred = [(Po & Polyhedron(eqns=[[0, 0, 0, 1], [e*(1-p), 0, 1, 0]])).linear_transformation(Matrix(ZZ, 1, 3, [[1, 0, 0]])) for Po in Pos]
        prop = maximum_degree(Pred, 1, p, e, 1)
        res = []
        for i in range(e):
            for g in pdiv:
                if (sum(res) + g) > prop:
                    res.append(g)
                    break
            else:
                res.append(p-1)
        pproperties.append((tuple(res), (p-1,)*e))
    else:
        res = []
        for i in range(e):
            for g in pdiv:
                if (sum(res) + g) > prop:
                    res.append(g)
                    break
            else:
                res.append(p-1)
        pproperties.append(((0,)*e, tuple(res)))
    print(r, degree[-1], pproperties[-1])
    if degree[-1] >= 2*e*(p-1)-1:
        break
    r += 1
    Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]

1 1 ((0, 0, 0, 0), (1, 0, 0, 0))
2 3 ((0, 0, 0, 0), (2, 0, 0, 0))
3 9 ((0, 0, 0, 0), (5, 0, 0, 0))
4 27 ((0, 0, 0, 0), (10, 0, 0, 0))
5 81 ((0, 0, 0, 0), (30, 0, 0, 0))
6 243 ((0, 0, 0, 0), (85, 0, 0, 0))
7 729 ((0, 0, 0, 0), (255, 0, 0, 0))
8 2187 ((0, 0, 0, 0), (771, 0, 0, 0))
9 6561 ((0, 0, 0, 0), (2570, 0, 0, 0))
10 19683 ((0, 0, 0, 0), (7710, 0, 0, 0))
11 59049 ((0, 0, 0, 0), (21845, 0, 0, 0))
12 146429 ((0, 0, 0, 0), (65535, 0, 0, 0))
13 223570 ((0, 0, 0, 0), (131070, 1, 0, 0))
14 249292 ((0, 0, 0, 0), (131070, 5, 0, 0))
15 257887 ((0, 0, 0, 0), (131070, 15, 0, 0))
16 260817 ((0, 0, 0, 0), (131070, 51, 0, 0))
17 261989 ((0, 0, 0, 0), (131070, 170, 0, 0))
18 262963 ((0, 0, 0, 0), (131070, 510, 0, 0))
19 265039 ((0, 0, 0, 0), (131070, 1285, 0, 0))
20 270986 ((0, 0, 0, 0), (131070, 3855, 0, 0))
21 288733 ((0, 0, 0, 0), (131070, 13107, 0, 0))
22 341941 ((0, 0, 0, 0), (131070, 43690, 0, 0))
23 429325 ((0, 0, 0, 0), (131070, 131070, 0, 0))
24 492630 ((0, 0, 0, 0), (131070, 131070, 1, 0

In [16]:
# for each round, check if p degree or q degree gives better properties
for r, (pprop, p2prop, qprop) in enumerate(zip(pproperties, p2properties, qproperties), start=1):
    pdata = product(pprop[0][i]+1 for i in range(e))*product(pprop[1][i]+1 for i in range(e))
    p2data = product(p2prop[0][i]+1 for i in range(2))*product(p2prop[1][i]+1 for i in range(2))
    qdata = (qprop[0]+1)*(qprop[1]+1)
    mdata = min(pdata, p2data, qdata)
    print(r, end=" ")
    if mdata == pdata:
        print(pprop, end=" ")
    if mdata == p2data:
        print(p2prop, end=" ")
    if mdata == qdata:
        print(qprop, end=" ")
    print()

1 ((0, 0, 0, 0), (1, 0, 0, 0)) ((0, 0), (1, 0)) (0, 1) 
2 ((0, 0, 0, 0), (2, 0, 0, 0)) ((0, 0), (2, 0)) (0, 2) 
3 ((0, 0), (4, 0)) (0, 4) 
4 ((0, 0, 0, 0), (10, 0, 0, 0)) ((0, 0), (10, 0)) (0, 10) 
5 ((0, 0, 0, 0), (30, 0, 0, 0)) ((0, 0), (30, 0)) (0, 30) 
6 ((0, 0, 0, 0), (85, 0, 0, 0)) ((0, 0), (85, 0)) (0, 85) 
7 ((0, 0, 0, 0), (255, 0, 0, 0)) ((0, 0), (255, 0)) (0, 255) 
8 ((0, 0), (768, 0)) (0, 768) 
9 (0, 2210) 
10 (0, 6630) 
11 (0, 19968) 
12 ((0, 0), (61440, 0)) (0, 61440) 
13 (0, 179605) 
14 (0, 532480) 
15 (0, 1597440) 
16 (0, 5003344) 
17 (0, 14483456) 
18 (0, 43274240) 
19 (0, 129822720) 
20 (0, 390260832) 
21 (0, 1163202560) 
22 (0, 3489607680) 
23 (0, 10572065872) 
24 (0, 31716197616) 
25 (0, 94164746240) 
26 (0, 282494238720) 
27 (0, 849069040345) 
28 (0, 2547207121035) 
29 (0, 7865110429696) 
30 (0, 23004815337472) 
31 (0, 69014446012416) 
32 (0, 210675205406720) 
33 (0, 632025616220160) 
34 (0, 2054083252715520) 
35 (0, 5564458862804992) 
36 (0, 16693376588414976) 
37 

In [17]:
# degree and properties mod p^2 with q-degrees
p = 2**17-1
e = 4
d = 3
pdiv = divisors(p**e-1)

P_f = get_polytope_function(p**e, 1, d)
P_round = feistel(P_f, p**e)
Pos = [(P_round & Polyhedron(eqns = [[0, 0, 0, 1, 0, 0]], ieqs=[[p**i, 0, 0, 0, -1, 0]])).linear_transformation(Matrix(ZZ, 3, 5, [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]])) for i in range(e)]
Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]
Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]
Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]
Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]
r = 5 # skipping the first 4 rounds do to problems with the PPL solver
degree = []
qproperties = []
while True:
    # we focus on the right branch, because this branch has the slower degree growth
    ## fix output property and divisibility
    Preds = [(Po & Polyhedron(ieqs=[[1, 0, 0, -1]])) for Po in Pos]
    ## compute degree
    degree.append(maximum_degree_with_divisibility(Preds, 2, p, e, e, 1))
    ## properties
    Pred = [(Po & Polyhedron(eqns=[[0, 1, 0, 0]], ieqs=[[1, 0, 0, -1]])).linear_transformation(Matrix(ZZ, 2, 3, [[0, 1, 0], [0, 0, 1]])) for Po in Pos]
    prop = maximum_degree_with_divisibility(Pred, 1, p, e, e, 1)
    if prop == p**e-1:
        Pred = [(Po & Polyhedron(eqns=[[1-p**e, 0, 1, 0]], ieqs=[[1, 0, 0, -1]])).linear_transformation(Matrix(ZZ, 2, 3, [[1, 0, 0], [0, 0, 1]])) for Po in Pos]
        prop = maximum_degree_with_divisibility(Pred, 1, p, e, e, 1)
        for g in pdiv:
            if g > prop:
                qproperties.append((g, p**e-1)) # left branch, right branch
                break
    else:
        for g in pdiv:
            if g > prop:
                qproperties.append((0, g))
                break
    print(r, degree[-1], qproperties[-1])
    if degree[-1] >= 2*(p**e-1)-1:
        break
    r += 1
    Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]

5 590273292669804609617 (0, 295138898083176775680)
6 590273292669804609779 (0, 295138898083176775680)
7 590273292669804610265 (0, 295138898083176775680)
8 590273292669804611723 (0, 295138898083176775680)
9 590273292669804616097 (0, 295138898083176775680)
10 590273292669804629219 (0, 295138898083176775680)
11 590273292669804668585 (0, 295138898083176775680)
12 590273292669804786683 (0, 295138898083176775680)
13 590273292669805140977 (0, 295138898083176775680)
14 590273292669806203859 (0, 295138898083176775680)
15 590273292669809392505 (0, 295138898083176775680)
16 590273292669818958443 (0, 295138898083176775680)
17 590273292669847656257 (0, 295138898083176775680)
18 590273292669933749699 (0, 295138898083176775680)
19 590273292670192030025 (0, 295138898083176775680)
20 590273292670966871003 (0, 295138898083176775680)
21 590273292673291393937 (0, 295138898083176775680)
22 590273292680264962739 (0, 295138898083176775680)
23 590273292701185669145 (0, 295138898083176775680)
24 59027329276394

In [18]:
# degree and properties mod p^2 with p^2-degrees
p = 2**17-1
e = 4
d = 3
s = 2

pdiv = [0] + divisors(p**s-1)

P_f = get_polytope_function(p**e, 1, d)
P_round = feistel(P_f, p**e)
Pos = [(P_round & Polyhedron(eqns = [[0, 0, 0, 1, 0, 0]], ieqs=[[p**i, 0, 0, 0, -1, 0]])).linear_transformation(Matrix(ZZ, 3, 5, [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]])) for i in range(e)]
Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]
Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]
Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]
Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]
r = 5 # skipping the first 4 rounds do to problems with the PPL solver
degree = []
p2properties = []
while True:
    # we focus on the right branch, because this branch has the slower degree growth
    ## fix output property and divisibility
    Preds = [(Po & Polyhedron(ieqs=[[1, 0, 0, -1]])) for Po in Pos]
    ## compute degree
    degree.append(maximum_degree_with_divisibility(Preds, 2, p, e, s, 1))
    ## properties
    Pred = [(Po & Polyhedron(eqns=[[0, 1, 0, 0]], ieqs=[[1, 0, 0, -1]])).linear_transformation(Matrix(ZZ, 2, 3, [[0, 1, 0], [0, 0, 1]])) for Po in Pos]
    prop = maximum_degree_with_divisibility(Pred, 1, p, e, s, 1)
    if prop == e/s*(p**s-1):
        Pred = [(Po & Polyhedron(eqns=[[e/s*(1-p**s), 0, 1, 0]], ieqs=[[1, 0, 0, -1]])).linear_transformation(Matrix(ZZ, 2, 3, [[1, 0, 0], [0, 0, 1]])) for Po in Pos]
        prop = maximum_degree_with_divisibility(Pred, 1, p, e, s, 1)
        res = []
        for i in range(e/s):
            for g in pdiv:
                if (sum(res) + g) > prop:
                    res.append(g)
                    break
            else:
                res.append(p**s-1)
        p2properties.append((tuple(res), (p**s-1,)*(e//s)))
    else:
        res = []
        for i in range(e/s):
            for g in pdiv:
                if (sum(res) + g) > prop:
                    res.append(g)
                    break
            else:
                res.append(p**s-1)
        p2properties.append(((0,)*(e//s), tuple(res)))
    print(r, degree[-1], p2properties[-1])
    if degree[-1] >= 2*e/s*(p**s-1)-1:
        break
    r += 1
    Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]

5 34358952017 ((0, 0), (17179607040, 0))
6 34358952179 ((0, 0), (17179607040, 0))
7 34358952665 ((0, 0), (17179607040, 0))
8 34358954123 ((0, 0), (17179607040, 0))
9 34358958497 ((0, 0), (17179607040, 0))
10 34358971619 ((0, 0), (17179607040, 0))
11 34359010985 ((0, 0), (17179607040, 0))
12 34359129083 ((0, 0), (17179607040, 0))
13 34359483377 ((0, 0), (17179607040, 49152))
14 34360546259 ((0, 0), (17179607040, 417792))
15 34363734905 ((0, 0), (17179607040, 1579008))
16 34373300843 ((0, 0), (17179607040, 5263360))
17 34401998657 ((0, 0), (17179607040, 15790080))
18 34488092099 ((0, 0), (17179607040, 44738560))
19 34746372425 ((0, 0), (17179607040, 134215680))
20 35521213403 ((0, 0), (17179607040, 429490176))
21 37845736337 ((0, 0), (17179607040, 1431633920))
22 44819305139 ((0, 0), (17179607040, 4294901760))
23 56272376501 ((0, 0), (17179607040, 17179607040))
24 64569569515 ((0, 0), (17179607040, 17179607040))
25 67335300529 ((0, 0), (17179607040, 17179607040))
26 68257210897 ((0, 0), 

In [19]:
# degree and properties mod p^2 with p-degrees
p = 2**17-1
e = 4
d = 3

pdiv = [0] + divisors(p-1)

P_f = get_polytope_function(p**e, 1, d)
P_round = feistel(P_f, p**e)
Pos = [(P_round & Polyhedron(eqns = [[0, 0, 0, 1, 0, 0]], ieqs=[[p**i, 0, 0, 0, -1, 0]])).linear_transformation(Matrix(ZZ, 3, 5, [[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 0, 0, 1]])) for i in range(e)]
Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]
Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]
Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]
Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]
r = 5 # skipping the first 4 rounds do to problems with the PPL solver
degree = []
pproperties = []
while True:
    # we focus on the right branch, because this branch has the slower degree growth
    ## fix output property and divisibility
    Preds = [(Po & Polyhedron(ieqs=[[1, 0, 0, -1]])) for Po in Pos]
    ## compute degree
    degree.append(maximum_degree_with_divisibility(Preds, 2, p, e, 1, 1))
    ## properties
    Pred = [(Po & Polyhedron(eqns=[[0, 1, 0, 0]], ieqs=[[1, 0, 0, -1]])).linear_transformation(Matrix(ZZ, 2, 3, [[0, 1, 0], [0, 0, 1]])) for Po in Pos]
    prop = maximum_degree_with_divisibility(Pred, 1, p, e, 1, 1)
    if prop == e*(p-1):
        Pred = [(Po & Polyhedron(eqns=[[e*(1-p), 0, 1, 0]], ieqs=[[1, 0, 0, -1]])).linear_transformation(Matrix(ZZ, 2, 3, [[1, 0, 0], [0, 0, 1]])) for Po in Pos]
        prop = maximum_degree_with_divisibility(Pred, 1, p, e, 1, 1)
        res = []
        for i in range(e):
            for g in pdiv:
                if (sum(res) + g) > prop:
                    res.append(g)
                    break
            else:
                res.append(p-1)
        pproperties.append((tuple(res), (p-1,)*e))
    else:
        res = []
        for i in range(e):
            for g in pdiv:
                if (sum(res) + g) > prop:
                    res.append(g)
                    break
            else:
                res.append(p**-1)
        pproperties.append(((0,)*e, tuple(res)))
    print(r, degree[-1], pproperties[-1])
    if degree[-1] >= 2*e*(p-1)-1:
        break
    r += 1
    Pos = [eval_output_property(P_round, Po, 2) for Po in Pos]

5 262217 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071))
6 262379 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071))
7 262865 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071))
8 264323 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071))
9 268697 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071))
10 281819 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071))
11 321185 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071))
12 408567 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071))
13 485709 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071))
14 511431 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071))
15 520026 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071))
16 522956 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071))
17 524127 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071))
18 525102 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071))
19 527178 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071))


In [20]:
# for each round, check if p degree or q degree gives better properties
for r, (pprop, p2prop, qprop) in enumerate(zip(pproperties, p2properties, qproperties), start=5):
    pdata = product(pprop[0][i]+1 for i in range(e))*product(pprop[1][i]+1 for i in range(e))
    p2data = product(p2prop[0][i]+1 for i in range(2))*product(p2prop[1][i]+1 for i in range(2))
    qdata = (qprop[0]+1)*(qprop[1]+1)
    mdata = min(pdata, p2data, qdata)
    print(r, end=" ")
    if mdata == pdata:
        print(pprop, end=" ")
    if mdata == p2data:
        print(p2prop, end=" ")
    if mdata == qdata:
        print(qprop, end=" ")
    print()

5 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071)) 
6 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071)) 
7 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071)) 
8 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071)) 
9 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071)) 
10 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071)) 
11 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071)) 
12 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071)) 
13 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071)) 
14 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071)) 
15 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071)) 
16 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071)) 
17 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071)) 
18 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071)) 
19 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071)) 
20 ((0, 0, 0, 0), (1/131071, 1/131071, 1/131071, 1/131071)) 
21 ((0, 0, 0, 0), (1/131071, 